# Custom CNN classifier
Try to build a custom CNN classifier for the dataset

### Dataloaders

In [4]:
import os
import torch
import torchvision

from pathlib import Path
from dotenv import load_dotenv

BATCH_SIZE = 8
LOADER_WORKERS = 8

load_dotenv()
root_data = os.getenv("KAGGLE_FILES_DIR")
dataset_path = Path(os.getcwd(), "..", root_data)
processed = Path(dataset_path, 'processed')


transformations = torchvision.transforms.Compose([
    torchvision.transforms.Resize((256, 256)),
    torchvision.transforms.ToTensor(),
])

train_dataset = torchvision.datasets.ImageFolder(root=str(Path(processed, 'train')), transform=transformations)
val_dataset = torchvision.datasets.ImageFolder(root=str(Path(processed, 'val')), transform=transformations)
test_dataset = torchvision.datasets.ImageFolder(root=str(Path(processed, 'test')), transform=transformations)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=LOADER_WORKERS)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=LOADER_WORKERS)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=LOADER_WORKERS)


### Size of datasets

In [5]:
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 64654
Validation dataset size: 18472
Test dataset size: 9238


### Define CNN classifier
Using CNN with 3 convolutional layers and 2 fully connected layers.

In [6]:
import torch
import torch.nn as nn


class CNN(nn.Module):
    def __init__(self, input_size: torch.Size, initial_filters: int, out_classes: int, dropout: float = 0.25, device: str = "cpu", ):
        super(CNN, self).__init__()
        channels, _, _ = input_size
        self.device = device
        self.conv = nn.Sequential(
            nn.Conv2d(channels, initial_filters, kernel_size=3, stride=1, padding=1, device=self.device),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(initial_filters, initial_filters * 2, kernel_size=3, stride=1, padding=1, device=self.device),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(initial_filters * 2, initial_filters * 4, kernel_size=3, stride=1, padding=1, device=self.device),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.perceptron = nn.Sequential(
            nn.Linear(self._get_conv_out_shape(input_size), initial_filters * 8, device=self.device),
            nn.Dropout(dropout),
            nn.Linear(initial_filters * 8, out_classes, device=self.device),
        )
    
    def forward(self, x: torch.Tensor):
        x = x.to(self.device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.perceptron(x)
        return x
    
    def _get_conv_out_shape(self, input_size: torch.Size):
        with torch.no_grad():
            zeros = torch.zeros(*input_size, device=self.device)
            z = self.conv(zeros)
            z = torch.prod(torch.tensor(z.shape))
        return z


### Early stopper

In [7]:
class EarlyStopper:
    def __init__(self, patience: int = 5, min_delta: float = 0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

### Evaluate function
For evaluation of the model, I will use accuracy and cross-entropy loss.

In [12]:
import numpy as np

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

def evaluate(
        model: nn.Module, 
        valid_loader: torch.utils.data.DataLoader, 
        loss_func: nn.Module, 
        epoch_no: int, 
        writer: SummaryWriter
):    
    model.eval()
    epoch_loss = 0
    correct_class = 0
    targets_list = []
    preds_list = []
    
    dataset_size = len(valid_loader.dataset)
    
    with torch.no_grad():
        device = model.device
        for inputs, targets in tqdm(valid_loader, desc="Evaluation: "):
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            pred_class = torch.argmax(output, dim=1)
            correct_class += (pred_class == targets).sum()
            loss = loss_func(output, targets)
            epoch_loss += loss.item() * inputs.size(0)
            
            # Count the number of targets and predictions
            targets_list.append(targets.cpu().numpy())
            preds_list.append(pred_class.cpu().numpy())
    
    avg_epoch_loss = epoch_loss / dataset_size
    accuracy = correct_class / dataset_size
    
    targets_np = np.concatenate(targets_list)
    preds_np = np.concatenate(preds_list)

    confusion_mat = confusion_matrix(targets_np, preds_np)
    precision = precision_score(targets_np, preds_np)
    recall = recall_score(targets_np, preds_np)
    f1 = f1_score(targets_np, preds_np)
    roc_auc = roc_auc_score(targets_np, preds_np)
    
    print(f"""Epoch: {epoch_no}
        Average epoch loss: {avg_epoch_loss}
        Confusion matrix: {confusion_mat}
        Accuracy: {accuracy}
        Precision: {precision:}
        Recall: {recall}
        F1: {f1}
        ROC AUC: {roc_auc}""")

    writer.add_scalar('Loss/train', avg_epoch_loss, epoch_no)
    writer.add_scalar('Accuracy/train', accuracy, epoch_no)
    writer.add_scalar('Precision/train', precision, epoch_no)
    writer.add_scalar('Recall/train', recall, epoch_no)
    writer.add_scalar('F1/train', f1, epoch_no)
    writer.add_scalar('ROC AUC/train', roc_auc, epoch_no)
    
    
    return avg_epoch_loss, accuracy, precision, recall, f1, roc_auc, confusion_mat

### Train function


In [13]:
import torch.optim as optim

def train(
        model: nn.Module, 
        train_loader: torch.utils.data.DataLoader,
        valid_loader: torch.utils.data.DataLoader,
        max_epochs: int,
        optimizer: optim.Optimizer, 
        loss_func: nn.Module,
        patience: int = 3,
        min_delta: float = 0.001,
):
    
    writer = SummaryWriter('../runs/baseline_lr01_delta001_100eps')
    device = model.device
    early_stopping = EarlyStopper(patience=patience, min_delta=min_delta)
    best_avg_loss = float('inf')
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    for epoch in range(1, max_epochs + 1):
        model.train()
        for inputs, targets in tqdm(train_loader, desc=f"Train epoch {epoch}: "):
            inputs = inputs.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
            predicted = model(inputs)
            loss = loss_func(predicted, targets)
            loss.backward()
            optimizer.step()
        
        avg_epoch_loss, accuracy, precision, recall, f1, roc_auc, confusion_mat = evaluate(model, valid_loader, loss_func, epoch, writer)
        
        lr_scheduler.step()
        
        if avg_epoch_loss < best_avg_loss:
            best_avg_loss = avg_epoch_loss
            torch.save(model.state_dict(), "../models/best_model_100eps.pt")
            print(f"Model saved on epoch {epoch}")
            
        if early_stopping.early_stop(avg_epoch_loss):
            print(f'''Early stopping on epoch {epoch}
            Validation loss: {avg_epoch_loss}
            Accuracy: {accuracy}
            Confusion Matrix: {confusion_mat}
            Precision: {precision}
            Recall: {recall}
            F1: {f1}
            ROC AUC: {roc_auc}''')
            break
    
    writer.close()


### Define training parameters

In [14]:
image, label = train_dataset[0]

DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

model_params = {
    "input_size": image.shape,
    "initial_filters": 8,
    "out_classes": len(train_dataset.classes),
    "device": DEVICE
}
cnn = CNN(**model_params)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.01)

n_epochs = 100

## Train custom model - establish baseline

In [ ]:
train(cnn, train_loader, val_loader, n_epochs, optimizer, loss_func)